# UniLMPrompt

可用的中文预训练参数：[`bert-base`](https://storage.googleapis.com/bert_models/2018_11_03/chinese_L-12_H-768_A-12.zip),[`roberta-wwm-ext-base`](https://drive.google.com/uc?export=download&id=1jMAKIJmPn7kADgD3yQZhpsqM-IRM1qZt),[`roberta-wwm-ext-large`](https://drive.google.com/uc?export=download&id=1dtad0FFzG11CBsawu8hvwwzU2R0FDI94),[`macbert-base`](https://drive.google.com/uc?export=download&id=1aV69OhYzIwj_hn-kO1RiBa-m8QAusQ5b),[`macbert-large`](https://drive.google.com/uc?export=download&id=1lWYxnk1EqTA2Q20_IShxBrCPc5VSDCkT)

In [1]:
import uf

print(uf.__version__)

v2.5.13


In [2]:
model = uf.UniLMPrompt("../../ref/bert_config.json", "../../ref/vocab.txt")
print(model)

INFO:tensorflow:LM Mode: `bi`. Use method `.to_mode()` to convert it into `bi`, `l2r`, `r2l` or `s2s`.
uf.UniLMPrompt(
    config_file="../../ref/bert_config.json",
    vocab_file="../../ref/vocab.txt",
    max_seq_length=128,
    init_checkpoint=None,
    output_dir=None,
    gpu_ids=None,
    drop_pooler=False,
    max_predictions_per_seq=20,
    masked_lm_prob=0.15,
    short_seq_prob=0.1,
    do_whole_word_mask=False,
    mode="bi",
    do_lower_case=True,
    truncate_method="LIFO",
)


In [4]:
# 语言模型：bi/l2r/r2l模式下
model.to_mode("bi")
X_tokenized = [
     [
          ["[Soft Prompt]"],                                            # soft prompt
          model.tokenizer.tokenize("被掩码的文字是？"),                    # task description
          model.tokenizer.tokenize("天亮以前说再见, 笑着泪流满面"),         # 文本
     ],
     [
          ["[Soft Prompt]"],                        
          model.tokenizer.tokenize("被掩码的文字是？"),                    
          model.tokenizer.tokenize("他想知道那是谁, 为何总沉默寡言"),
     ],
]
y = None

# 语言模型：s2s模式下
# model.to_mode("s2s")
# X = [["天亮以前说再见", "笑着泪流满面"], ["他想知道那是谁", "为何总沉默寡言"]]     # doc必须为两个句子

# 下游任务
X_tokenized = [
     [
          ["[Soft Prompt]"],                                                 # soft prompt
          ["情", "民"],                                                       # verbolizer prompt
          model.tokenizer.tokenize("下面这句歌词来自于") + ["[MASK]", "歌"],    # task description
          model.tokenizer.tokenize("天亮以前说再见, 笑着泪流满面"),              # 文本
     ],
     [
          ["[Soft Prompt]"],                        
          model.tokenizer.tokenize("被掩码的文字是？"),                    
          model.tokenizer.tokenize("他想知道那是谁, 为何总沉默寡言"),
     ],
]
y = None


# 训练

In [5]:
model.fit(X_tokenized=X_tokenized, total_steps=20)

/Users/geyingli/Library/Python/3.8/lib/python/site-packages/uf-2.5.13-py3.8.egg/uf/apps/bert/bert.py:235: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  query_layer = tf.layers.dense(
/Users/geyingli/Library/Python/3.8/lib/python/site-packages/uf-2.5.13-py3.8.egg/uf/apps/bert/bert.py:245: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  key_layer = tf.layers.dense(
/Users/geyingli/Library/Python/3.8/lib/python/site-packages/uf-2.5.13-py3.8.egg/uf/apps/bert/bert.py:255: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  value_layer = tf.layers.dense(
/Users/geyingli/Library/Python/3.8/lib/python/site-packages/uf-2.5.13-py3.8.egg/uf/apps/bert/bert.py:379: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Ple

INFO:tensorflow:Build graph with 307,461,528 parameters (among which 102,880,904 are trainable)
INFO:tensorflow:Running local_init_op
INFO:tensorflow:Done running local_init_op
INFO:tensorflow:Running training on 2 samples (step 0 -> 20)
INFO:tensorflow:step 1, MLM accuracy 0.0000, MLM loss 1.019995, 0.13 steps/sec, 0.25 examples/sec
INFO:tensorflow:step 2, MLM accuracy 0.7500, MLM loss 0.752626, 0.50 steps/sec, 1.01 examples/sec
INFO:tensorflow:step 3, MLM accuracy 1.0000, MLM loss 0.601650, 0.54 steps/sec, 1.07 examples/sec
INFO:tensorflow:step 4, MLM accuracy 1.0000, MLM loss 0.502519, 0.46 steps/sec, 0.93 examples/sec
INFO:tensorflow:step 5, MLM accuracy 1.0000, MLM loss 0.424412, 0.43 steps/sec, 0.86 examples/sec
INFO:tensorflow:step 6, MLM accuracy 1.0000, MLM loss 0.354928, 0.43 steps/sec, 0.86 examples/sec
INFO:tensorflow:step 7, MLM accuracy 1.0000, MLM loss 0.288862, 0.49 steps/sec, 0.99 examples/sec
INFO:tensorflow:step 8, MLM accuracy 1.0000, MLM loss 0.229064, 0.51 steps/s

# 推理

In [6]:
X_tokenized = [
     [
          ["[Soft Prompt]"],                                   # soft prompt
          model.tokenizer.tokenize("被掩码的文字是？"),           # hard prompt
          ["他", "想", "[MASK]", "道", "那", "是", "谁"],        # 手动赋予Mask (与训练阶段采样不同，因此此case预测错误正常)
     ],
]
model.predict(X_tokenized=X_tokenized)

INFO:tensorflow:Running inference on 1 samples
INFO:tensorflow:process 100.0%, 0.89 examples/sec


{'mlm_preds': [['何']]}